In [3]:
import scipy.io as sio
import pandas as pd
import numpy as np

from scipy import signal
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
!pip install neurokit2
import neurokit2 as nk
import matplotlib.pyplot as plt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.5 MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
raw = sio.loadmat("/content/drive/MyDrive/DREAMER.mat")

In [31]:
def preprocess(data):
  #for participant
  processed_data = np.zeros((23*18),dtype = list)
  i=0
  for p in range(0,23):

    age = data[0,p]["Age"][0][0][0]
    gender = data[0,p]["Gender"][0][0][0]
    #for video
    for v in range(0,18):

      #scores
      score_arousal = data[0,p]["ScoreArousal"][0,0][v][0]
      score_valence = data[0,p]["ScoreValence"][0,0][v][0]
      score_dominance = data[0,p]["ScoreDominance"][0,0][v][0]


      #ECG
      base_left = data [0, p]['ECG'][0, 0]['baseline'][0, 0][v, 0][:, 0]
      base_right = data [0, p]['ECG'][0, 0]['baseline'][0, 0][v, 0][:, 1]

      stim_left = data [0, p]['ECG'][0, 0]['stimuli'][0, 0][v, 0][:, 0]
      stim_right = data [0, p]['ECG'][0, 0]['stimuli'][0, 0][v, 0][:, 1]

      # process with neurokit
      signals_b_l, info_b_l = nk.ecg_process(base_left, 256)
      signals_b_r, info_b_r = nk.ecg_process(base_right, 256)
      signals_s_l, info_s_l = nk.ecg_process(stim_left, 256)
      signals_s_r, info_s_r = nk.ecg_process(stim_right, 256)

      features_ecg_l = nk.ecg_intervalrelated(signals_s_l)/ nk.ecg_intervalrelated(signals_b_l)
      features_ecg_r = nk.ecg_intervalrelated(signals_s_r)/ nk.ecg_intervalrelated(signals_b_r)
      
      # average left and right features
      features_ecg = (features_ecg_l + features_ecg_r)/2

  
      if gender == 'male':
        g = 1
      else:
        g = 0

      row = [p, int(age), g ,v,score_arousal, score_valence, score_dominance] 
      row = np.append(row,features_ecg)

      #EEG 
      eeg = []

      
      for c in range(0,14):

        base = data[0, p]['EEG'][0, 0]['baseline'][0,0][v, 0][:, c]
        stim = data[0, p]['EEG'][0, 0]['stimuli'][0,0][v, 0][:, c]

        # feature extraction - PSD
        #filtering 

        b, a = signal.iirnotch(50, 30, 128)
        base = signal.filtfilt(b,a,base)
        stim = signal.filtfilt(b,a,stim)

        #theta
        b, a = signal.butter(4, [4/128, 8/128], 'bandpass', analog=True)
        base_theta_filt = signal.filtfilt(b,a,base)
        stim_theta_filt = signal.filtfilt(b,a,stim)

        #alpha
        b, a = signal.butter(4, [8/128, 12/128], 'bandpass', analog=True)
        base_alpha_filt = signal.filtfilt(b,a,base)
        stim_alpha_filt = signal.filtfilt(b,a,stim)

        #beta
        b, a = signal.butter(4, [13/128, 30/128], 'bandpass', analog=True)
        base_beta_filt = signal.filtfilt(b,a,base)
        stim_beta_filt = signal.filtfilt(b,a,stim)

        #features
        #baseline
        ftheta, bpsdtheta = signal.periodogram(base_theta_filt, 128)
        falpha, bpsdalpha = signal.periodogram(base_alpha_filt, 128)
        fbeta, bpsdbeta = signal.periodogram(base_beta_filt, 128)

        #stimuli
        ftheta, spsdtheta = signal.periodogram(stim_theta_filt, 128)
        falpha, spsdalpha = signal.periodogram(stim_alpha_filt, 128)
        fbeta, spsdbeta = signal.periodogram(stim_beta_filt, 128)

        features_theta = np.divide(max(spsdtheta),max(bpsdtheta))
        features_alpha = np.divide(max(spsdalpha),max(bpsdalpha))
        features_beta = np.divide(max(spsdbeta),max(bpsdbeta))

        #print(row)
        eeg.append(features_theta)
        eeg.append(features_alpha)
        eeg.append(features_beta)

        
      processed_data[i] = np.append(row,eeg)
      i += 1
  return processed_data


In [ ]:
data = raw["DREAMER"][0,0]["Data"]
processed_data = preprocess(data)

In [34]:
print(processed_data)

[array([ 0.00000000e+00,  2.20000000e+01,  1.00000000e+00,  0.00000000e+00,
         3.00000000e+00,  4.00000000e+00,  2.00000000e+00,  1.01935013e+00,
         9.80714795e-01,  9.42895037e-01,             nan,             nan,
                    nan,             nan,             nan,             nan,
         9.37462856e-01,  9.33189040e-01,  9.61436447e-01,  9.55897162e-01,
         9.81729930e-01,  1.00000000e+00,  1.01861702e+00,  1.15384615e+00,
         9.78260870e-01,  9.94974874e-01,             nan,             inf,
         9.67938256e-01,  1.01126126e+00,  8.91760367e-01,  1.04166667e+00,
                    nan,             nan,             nan,  7.49677320e-01,
                    nan,             nan,             nan,  6.27252816e-01,
         1.12466740e+00,  9.33189040e-01,  9.43994706e-01,  9.88531176e-01,
         8.80949962e-01,  1.01169504e+00,  9.82182580e-01,  9.55013426e-01,
         1.02471596e+00,  9.83000152e-01,  8.91429759e-01,  1.38068182e+00,
         1.0

In [35]:
processed_data = pd.DataFrame(processed_data)

In [36]:
processed_data

,0
0,"[0.0, 22.0, 1.0, 0.0, 3.0, 4.0, 2.0, 1.0193501..."
1,"[0.0, 22.0, 1.0, 1.0, 3.0, 3.0, 1.0, 0.9019112..."
2,"[0.0, 22.0, 1.0, 2.0, 4.0, 5.0, 4.0, 1.0327772..."
3,"[0.0, 22.0, 1.0, 3.0, 3.0, 4.0, 2.0, 0.9521824..."
4,"[0.0, 22.0, 1.0, 4.0, 4.0, 4.0, 4.0, 1.0632386..."
...,...
409,"[22.0, 25.0, 1.0, 13.0, 2.0, 2.0, 2.0, 0.92177..."
410,"[22.0, 25.0, 1.0, 14.0, 2.0, 2.0, 2.0, 1.06203..."
411,"[22.0, 25.0, 1.0, 15.0, 3.0, 3.0, 2.0, 0.98130..."
412,"[22.0, 25.0, 1.0, 16.0, 2.0, 2.0, 4.0, 0.96412..."


In [37]:
processed_data.columns
df = processed_data

In [38]:
print(df.columns)

RangeIndex(start=0, stop=1, step=1)


In [39]:
info = np.zeros((414,139))
for i in range(0,414):
  info[i,:] = df.iloc[i][0]

final = pd.DataFrame(info)
display(final)

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,0.0,22.0,1.0,0.0,3.0,4.0,2.0,1.019350,0.980715,0.942895,...,0.641331,3.046169,3.046303,3.047612,0.433049,0.433089,0.433394,0.386359,0.386398,0.386690
1,0.0,22.0,1.0,1.0,3.0,3.0,1.0,0.901911,1.108991,1.212197,...,1.817823,1.118235,1.118234,1.118254,1.127221,1.127238,2.211942,0.080668,0.080667,0.211553
2,0.0,22.0,1.0,2.0,4.0,5.0,4.0,1.032777,0.967888,1.094067,...,2.976669,4.070604,4.070590,4.070283,4.956444,4.956275,3.298131,7.438579,7.438391,3.140970
3,0.0,22.0,1.0,3.0,3.0,4.0,2.0,0.952182,1.049724,1.190338,...,0.595770,1.781350,1.781311,1.676477,0.539994,0.540057,0.540550,0.530569,0.530601,0.530851
4,0.0,22.0,1.0,4.0,4.0,4.0,4.0,1.063239,0.942383,0.685394,...,1.252920,6.574025,6.573898,5.230920,1.518394,1.518434,1.519098,1.498613,1.498624,1.498986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,22.0,25.0,1.0,13.0,2.0,2.0,2.0,0.921778,1.085352,0.811779,...,0.824809,5.303738,5.303693,5.303463,1.749653,1.749745,1.750785,3.001060,3.001071,3.001074
410,22.0,25.0,1.0,14.0,2.0,2.0,2.0,1.062034,0.941156,0.805474,...,2.439457,6.230672,6.230659,6.230196,0.602601,0.602605,0.602632,3.284671,3.284683,3.285313
411,22.0,25.0,1.0,15.0,3.0,3.0,2.0,0.981307,1.019091,0.769250,...,0.242219,0.609572,0.609569,0.609673,0.227857,0.227840,0.227921,6.720858,6.720897,6.719676
412,22.0,25.0,1.0,16.0,2.0,2.0,4.0,0.964130,1.035855,0.729928,...,0.139308,0.603502,0.603496,0.603458,0.047192,0.047191,0.047188,2.435460,2.435472,2.435429


In [40]:
c1=["age", "gender", "participant","video", "valence", "arousal", "dominance"]
col = []
for i in range(0, 14):
  col.append("psdtheta_"+str(i + 1)+"_un")
  col.append("psdalpha_"+str(i + 1)+"_un")
  col.append("psdbeta_"+str(i + 1)+"_un")

ek = []
for i in range(0,90):
  ek.append('EKG_'+str(i))

columns = c1+ek+col
print(len(columns))

139


In [41]:
final.columns = columns

In [42]:
final.to_csv('data_final.csv')

In [ ]:
display(final)